In [1]:
%matplotlib inline
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import os
from scipy import spatial
from collections import defaultdict
tf.logging.set_verbosity(tf.logging.ERROR)
import datetime
import timeit 


/Users/nathanielschub/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/nathanielschub/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [5]:
#os.chdir('..')
!pwd
sent_1= pd.read_csv('Code/sentence_generator/output.txt', sep = ',', names = ['pair', 'word', 'sentence'])
sent_2= pd.read_csv('Code/sentence_generator/output_2.txt', sep = ',', names = ['pair', 'word', 'sentence'])

/Users/nathanielschub/Desktop/MIDS/W266/smartThesaurus


In [11]:
distances = []       
start = timeit.default_timer()
for i in range(500, len(np.unique(sent_1.pair))+1):
#for i in range(1, 500):
    pair_no = i

    sents_1= np.asarray(sent_1.loc[sent_1['pair'] == i].sentence)
    word_1 = np.unique(sent_1.loc()[sent_1['pair'] == i].word)[0]
    word_1_indices = []
    for sent in sents_1:
        x = sent.split()
        word_1_indices.append(x.index(word_1))

    sents_2 = np.asarray(sent_2.loc[sent_2['pair'] == i].sentence)
    word_2 = np.unique(sent_2.loc()[sent_1['pair'] == i].word)[0]
    word_2_indices = []
    for sent in sents_2:
        x = sent.split()
        word_2_indices.append(x.index(word_2))


    embeddings_1 = elmo(sents_1,
    signature="default",
    as_dict=True)["elmo"] 

    embeddings_2 = elmo(sents_2,
    signature="default",
    as_dict=True)["elmo"] 

    init = tf.initialize_all_variables()
    sess = tf.Session()
    sess.run(init)
    embed_1 = sess.run(embeddings_1)
    embed_2 = sess.run(embeddings_2)

    target_embeddings_1 = []
    for i in range(len(embed_1)):
        target_embeddings_1.append(embed_1[i][word_1_indices[i]])
        mean_embedding_1 = np.mean(np.asarray(target_embeddings_1), axis = 0)

    target_embeddings_2 = []
    for i in range(len(embed_2)):
        target_embeddings_2.append(embed_2[i][word_2_indices[i]])
        mean_embedding_2 = np.mean(np.asarray(target_embeddings_2), axis = 0)

    distance = spatial.distance.cosine(mean_embedding_1, mean_embedding_2)
    distances.append((pair_no, distance))

    if pair_no%5==0:
        current_time = timeit.default_timer()
        delta = current_time - start
        print(pair_no,':', delta)
        


5 : 22.4971685372293


In [12]:
print(distances)

[(1, 0.25053220987319946), (2, 0.2510792016983032), (3, 0.508088618516922), (4, 0.342008113861084), (5, 0.5835147798061371), (6, 0.4221990704536438)]
